# Objective
Create a customer service agent with client side tools. The chatbot can:
- lookup information
- retrieve order details
- cancel orders



# How to use tools

1. Provide tools in user prompt

2. Claude decides to use a tool

3. Extract tool input, run code, and return results

4. Claude uses tool result to formulate a response

Note steps 3 and 4 are optional.

**Specifying tools**
- name
- description
- input_schema: json schema defining the expected parameters


**Specifying a prompt for tool use**

```
In this environment you have access to a set of tools you can use to answer the user's question.
{{ FORMATTING INSTRUCTIONS }}
String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular expressions.
Here are the functions available in JSONSchema format:
{{ TOOL DEFINITIONS IN JSON SCHEMA }}
{{ USER SYSTEM PROMPT }}
{{ TOOL CONFIGURATION }}
```

**Tips**
- Provide extremely detailed desrciptions
    - what the tool does
    - when it should be used
    - what each parameter means and how it affects tool´s behavior
    - caveats or limitations
- Prioritize descriptions over examples

**Forcing tool use**

Use:
```
tool_choice = {"type": "tool", "name": "get_weather"}
```

### Step 1: Define model and setup environment

In [1]:
from dotenv import load_dotenv
import os
import anthropic

load_dotenv()
client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

In [2]:
MODEL_NAME = "claude-3-opus-20240229"

### Step 2: Define the client-side tools


In [4]:
tools = [
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

### Step 3: Simulate synthetic tool responses

In real life, these functions are replaced with your business database

In [5]:
def get_customer_info(customer_id):
    # Simulated customer data
    customers = {
        "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
        "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
    }
    return customers.get(customer_id, "Customer not found")

def get_order_details(order_id):
    # Simulated order data
    orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }
    return orders.get(order_id, "Order not found")

def cancel_order(order_id):
    # Simulated order cancellation
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False

### Step 4: Process tool calls and return results

In [7]:
def process_tool_call(tool_name, tool_input):
    
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

### Step 5: interact with the chatbot

In [8]:
import json

def chatbot_interaction(user_message):

    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    # When Claude decides to use one of the tools you’ve provided, it will return a response 
    # with a stop_reason of tool_use and one or more tool_use content blocks in the API response
    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        # Provide last interaction as context
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response

### Step 6: test it!

In [9]:
chatbot_interaction("Can you tell me the email address for customer C1?")
chatbot_interaction("What is the status of order O2?")
chatbot_interaction("Please cancel order O1 for me.")


User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='<thinking>\nTo find the email address for customer C1, I will need to use the get_customer_info tool. \n\nThe get_customer_info tool requires the following parameter:\n- customer_id (required)\n\nThe user provided the customer ID in their request - "C1". So I have the necessary information to make this tool call.\n\nNo other tools are needed to answer this request. I will proceed with calling the get_customer_info tool with the customer_id parameter set to "C1".\n</thinking>', type='text'), ToolUseBlock(id='toolu_01H6x9MgmmSnsEqMvKEAFTcK', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Tool Used: get_customer_info
Tool Input:
{
  "customer_id": "C1"
}

Tool Result:
{
  "name": "John Doe",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(text='The email address for cu

"Great, I've successfully cancelled your order O1. Please let me know if there's anything else I can assist you with."